KeyError: "['RIAGENDER', 'anemia'] not in index"

In [2]:
import pandas as pd

# Load files
demo = pd.read_sas("DEMO_J.XPT")
dbq  = pd.read_sas("DBQ_J.XPT")
rhq  = pd.read_sas("RHQ_J.XPT")
mcq  = pd.read_sas("MCQ_J.XPT")
cbc  = pd.read_sas("CBC_J.XPT")


In [4]:
df = demo.merge(dbq, on="SEQN", how="left") \
         .merge(rhq, on="SEQN", how="left") \
         .merge(mcq, on="SEQN", how="left") \
         .merge(cbc, on="SEQN", how="left")


In [5]:
df["anemia"] = 0

# Women: Hb < 12
df.loc[(df["RIAGENDR"] == 2) & (df["LBXHGB"] < 12), "anemia"] = 1

# Men: Hb < 13
df.loc[(df["RIAGENDR"] == 1) & (df["LBXHGB"] < 13), "anemia"] = 1


In [6]:
df = df[df["LBXHGB"].notna()]


In [7]:
selected_columns = [
    "SEQN",
    "RIDAGEYR",   # age
    "RIAGENDR",   # gender
    "RIDRETH1",   # ethnicity
    "INDFMPIR",   # income ratio
    "LBXHGB",     # hemoglobin
    "anemia"
]

df_clean = df[selected_columns]


In [8]:
df_clean.to_csv("nhanes_anemia_dataset.csv", index=False)


In [9]:
print(dbq.columns)
print(rhq.columns)
print(mcq.columns)


Index(['SEQN', 'DBQ010', 'DBD030', 'DBD041', 'DBD050', 'DBD055', 'DBD061',
       'DBQ073A', 'DBQ073B', 'DBQ073C', 'DBQ073D', 'DBQ073E', 'DBQ073U',
       'DBQ700', 'DBQ197', 'DBQ223A', 'DBQ223B', 'DBQ223C', 'DBQ223D',
       'DBQ223E', 'DBQ223U', 'DBQ229', 'DBQ235A', 'DBQ235B', 'DBQ235C',
       'DBQ301', 'DBQ330', 'DBQ360', 'DBQ370', 'DBD381', 'DBQ390', 'DBQ400',
       'DBD411', 'DBQ421', 'DBQ424', 'DBD895', 'DBD900', 'DBD905', 'DBD910',
       'CBQ596', 'CBQ606', 'CBQ611', 'DBQ930', 'DBQ935', 'DBQ940', 'DBQ945'],
      dtype='object')
Index(['SEQN', 'RHQ010', 'RHD018', 'RHQ020', 'RHQ031', 'RHD043', 'RHQ060',
       'RHQ070', 'RHQ074', 'RHQ076', 'RHQ078', 'RHQ131', 'RHD143', 'RHQ160',
       'RHQ162', 'RHQ163', 'RHQ166', 'RHQ169', 'RHQ172', 'RHD173', 'RHQ171',
       'RHD180', 'RHD190', 'RHQ197', 'RHQ200', 'RHD280', 'RHQ291', 'RHQ305',
       'RHQ332', 'RHQ420', 'RHQ540', 'RHQ542A', 'RHQ542B', 'RHQ542C',
       'RHQ542D', 'RHQ554', 'RHQ560Q', 'RHQ560U', 'RHQ570', 'RHQ576Q',
       '

In [ ]:
import pandas as pd

# =========================
# 1) LOAD DATASETS
# =========================
demo = pd.read_sas("DEMO_J.XPT")
cbc  = pd.read_sas("CBC_J.XPT")
dbq  = pd.read_sas("DBQ_J.XPT")
rhq  = pd.read_sas("RHQ_J.XPT")
mcq  = pd.read_sas("MCQ_J.XPT")

# =========================
# 2) MERGE ON SEQN
# =========================
df = demo.merge(cbc, on="SEQN", how="inner")
df = df.merge(dbq, on="SEQN", how="left")
df = df.merge(mcq, on="SEQN", how="left")



print("Merged shape:", df.shape)

# =========================
# 3) REMOVE INCOME COLUMN
# =========================
if "INDFMPIR" in df.columns:
    df = df.drop(columns=["INDFMPIR"])

# =========================
# 4) DETECT GENDER COLUMN
# =========================
if "RIAGENDR" in df.columns:
    gender_col = "RIAGENDR"
elif "RIAGENDER" in df.columns:
    gender_col = "RIAGENDER"
else:
    raise Exception("Gender column not found!")

# =========================
# 5) CREATE ANEMIA LABEL
# =========================
def calculate_anemia(row):
    if pd.isna(row["LBXHGB"]):
        return None
    
    # Male = 1, Female = 2 in NHANES
    if row[gender_col] == 1 and row["LBXHGB"] < 13:
        return 1
    elif row[gender_col] == 2 and row["LBXHGB"] < 12:
        return 1
    else:
        return 0

df["anemia"] = df.apply(calculate_anemia, axis=1)

# =========================
# 6) CANDIDATE QUESTION FEATURES
# (We will automatically pick only those that exist)
# =========================
candidate_questions = [
    # Demographics
    "RIDAGEYR",
    gender_col,
    "RIDRETH1",

    # Diet (replace mostly empty columns)
    "DBQ197",    # Fruit intake frequency
    "DBQ235A",   # Vegetable intake frequency
    "DBQ229",    # Water / sugar drink intake
    "DBQ223A",   # Keep this one, usually filled

    # Reproductive / supplements
    "RHQ010",
    "RHQ020",
    "RHQ060",

    # Medical conditions (replace mostly empty ones)
    "MCQ010",    # Diabetes
    "MCQ053",    # Hypertension
    "MCQ080",    # Heart disease
    "MCQ160A",   # Asthma
    "MCQ025",    # Cancer (better filled)
    "MCQ040"     # Arthritis (better filled)
]


# =========================
# 7) KEEP ONLY EXISTING COLUMNS
# =========================
existing_features = [col for col in candidate_questions if col in df.columns]

# Limit to maximum 15 questionnaire features
existing_features = existing_features[:15]

# Add hemoglobin + anemia label
final_features = ["SEQN"] + existing_features + ["LBXHGB", "anemia"]
df_final = df[final_features]


# =========================
# 8) DROP ROWS WITH MISSING LABEL
# =========================
df_final = df_final.dropna(subset=["anemia"])

print("Final dataset shape:", df_final.shape)
print("Selected features:")
print(df_final.columns)

# =========================
# 9) SAVE TO CSV
# =========================
df_final.to_csv("anemia_questionnaire_dataset.csv", index=False)

print("Dataset saved successfully.")


Merged shape: (8366, 187)
Final dataset shape: (7528, 16)
Selected features:
Index(['SEQN', 'RIDAGEYR', 'RIAGENDR', 'RIDRETH1', 'DBQ010', 'DBD030',
       'DBQ073A', 'DBQ223A', 'MCQ010', 'MCQ053', 'MCQ080', 'MCQ160A',
       'MCQ510A', 'MCQ050', 'LBXHGB', 'anemia'],
      dtype='object')
Dataset saved successfully.


In [9]:
print(df_final["anemia"].value_counts())
print(df_final["anemia"].value_counts(normalize=True))


anemia
0.0    6331
1.0    1197
Name: count, dtype: int64
anemia
0.0    0.840994
1.0    0.159006
Name: proportion, dtype: float64
